In [ ]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, firestore
import requests
import pandas as pd

In [ ]:
# Inisialisasi Firebase Admin SDK
cred = credentials.Certificate('/work/staklimjerukagung-firebase-admin.json')  # ganti dengan path ke service account key
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [ ]:
# Referensi Firestore
firestore_db = firestore.client()

In [ ]:
# URL Firebase RTDB untuk mengambil data
url_rtdb = 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/auto_weather_stat/id-03/data.json'

In [2]:
# Ambil data dari Firebase RTDB
response = requests.get(url_rtdb)
if response.status_code == 200:
    rtdb_data = response.json()

    # Convert RTDB data ke DataFrame pandas
    df = pd.DataFrame.from_dict(rtdb_data, orient='index')
    df['timestamp'] = df.index  # Memindahkan timestamp dari index ke kolom

    print("Data dari RTDB berhasil dibaca:")
    print(df.head())  # Print beberapa baris untuk verifikasi

    # Pindahkan data ke Firestore dengan struktur sensor_iklim -> id-01 -> data
    collection_ref = firestore_db.collection('sensor_iklim').document('id-01').collection('data')

    for _, row in df.iterrows():
        collection_ref.document(row['timestamp']).set(row.to_dict())  # Simpan setiap baris data

    print("Data berhasil dipindahkan ke Firestore.")
else:
    print("Gagal mengambil data dari RTDB:", response.status_code)

Data dari RTDB berhasil dibaca:
              dew humidity pressure temperature   timestamp  volt
0           22.62    89.33  1011.29       24.49           0  4.06
1702296831  23.29    89.28  1010.72       25.18  1702296831  4.08
1702296891  23.29    89.38  1010.75       25.16  1702296891  4.08
1702296951  23.31    89.58  1010.76       25.14  1702296951  4.08
1702297011  23.35    89.86  1010.82       25.13  1702297011  4.08


RetryError: Deadline of 60.0s exceeded while calling target function, last exception: 429 Quota exceeded.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>